# Monad Exercises

## IGNORE THIS (Setup)


In [9]:
:set -XNoImplicitPrelude
:set -XTypeSynonymInstances
:set -XInstanceSigs
:l ../../original-source/Course/Core.hs
:l ../../original-source/Course/ExactlyOne.hs
:l ../../original-source/Course/Optional.hs
:l ../../original-source/Course/List.hs
:l ../../original-source/Course/Functor.hs
:l ../../original-source/Course/Applicative.hs
import qualified Prelude as P
import Prelude (String)
import Data.Foldable (for_, traverse_)
import Test.QuickCheck

runTests :: [(P.String, Property)] -> IO ()
runTests tests = for_ tests $ \(name, prop) -> do
  P.putStrLn $ "Testing: " P.++ name
  quickCheck prop

threeOf :: Gen a -> Gen (a, a, a)
threeOf g = do
  x <- g
  y <- g
  z <- g
  P.return (x, y, z)

listOf :: Gen a -> Gen (List a)
listOf g =
  let
    genN 0 = P.return Nil
    genN n = do
      h <- g
      t <- genN (n - 1)
      P.return (h :. t)
  in do
    len <- choose (0, 100)
    genN len

## Monad

### Definition

In [10]:
class (Applicative f) => Monad f where
--mappend :: a         -> a    -> a
  (>>=)   :: f a -> (a -> f b) -> f b

infixl 1 >>=

(=<<) :: (Monad m) => (a -> m b) -> m a -> m b
f =<< mx = mx >>= f

infixr 1 =<<

### Laws

In **addition** to the `Functor` and `Applicative` functions and laws, Monad has the following laws, which are
**again** similar to the `Monoid` laws.

Also the `>>=/=<</join` "addition" functions for a `Monad` must be consistent with the addition function `<*>` in `Applicative`, i.e

``` haskell
f <$> fx <*> fy = fx >>= \x -> fy >>= \y -> pure (f x y)
```

#### Law of Associativity

``` haskell
(mx >>= g) >>= f === mx >>= (\x -> g x >>= f)
```

## Exercises

### 1. Write a `Monad` Instance for `ExactlyOne`

#### Exercise

In [11]:
instance Monad ExactlyOne where
  (>>=) :: ExactlyOne a -> (a -> ExactlyOne b) -> ExactlyOne b
  (>>=) = _bind

#### Try it out

In [7]:
-- >>> (\x -> ExactlyOne(x+1)) =<< ExactlyOne 2
-- ExactlyOne 3
ExactlyOne 2 >>= \x -> ExactlyOne (x + 1)

ExactlyOne 3

#### Tests

In [17]:
exactlyOneTest :: Property
exactlyOneTest = once $
  (ExactlyOne 2 >>= \x -> ExactlyOne (x + 1)) === ExactlyOne 3

exactlyOneTests :: IO ()
exactlyOneTests = runTests [
    ("ExactlyOne 2 >>= \\x -> ExactlyOne (x + 1)", exactlyOneTest)
  ]

exactlyOneTests

Testing: ExactlyOne 2 >>= \x -> ExactlyOne (x + 1)
+++ OK, passed 1 tests.

### 2. Write a `Monad` Instance for `List`

#### Exercise

In [19]:
instance Monad List where
  (>>=) :: List a -> (a -> List b) -> List b
  (>>=) = _bind

#### Try it out

In [20]:
-- >>> (\n -> n :. n :. Nil) =<< (1 :. 2 :. 3 :. Nil)
-- [1,1,2,2,3,3]
(1 :. 2 :. 3 :. Nil) >>= (\n -> n :. n :. Nil)

[1,1,2,2,3,3]

#### Tests

In [22]:
listMonadTest :: Property
listMonadTest = once $
  ((1 :. 2 :. 3 :. Nil) >>= (\n -> n :. n :. Nil)) === (1 :. 1 :. 2 :. 2:. 3 :. 3 :. Nil)

runListTests :: IO ()
runListTests = runTests [
    ("(1 :. 2 :. 3 :. Nil) >>= (\\n -> n :. n :. Nil)", listMonadTest)
  ]

runListTests

Testing: (1 :. 2 :. 3 :. Nil) >>= (\n -> n :. n :. Nil)
+++ OK, passed 1 tests.

### 3. Write `Monad` instance for `Optional`

#### Exercise

In [23]:
instance Monad Optional where
  (>>=) :: Optional a -> (a -> Optional b) -> Optional b
  (>>=) = _bind

#### Try it out

In [24]:
-- >>> (\n -> Full (n + n)) =<< Full 7
-- Full 14
Full 7 >>= \n -> Full (n + n)

Full 14

#### Tests

In [26]:
optionalEmptyTest :: Property
optionalEmptyTest = once $
  (Empty >>= (\n -> Full (2 * n))) === Empty

optionalFullTest :: Property
optionalFullTest = forAll arbitrary $ \n ->
  (Full n >>= \n -> Full (2 * n)) === Full (2 * n)

runOptionalTests :: IO ()
runOptionalTests = runTests [
    ("(Empty >>= (\\n -> Full (2 * n))) === Empty", optionalEmptyTest)
  , ("forall n. (Full n >>= \\n -> Full (2 * n)) === Full (2 * n)", optionalFullTest)
  ]

runOptionalTests

Testing: (Empty >>= (\n -> Full (2 * n))) === Empty
+++ OK, passed 1 tests.
Testing: forall n. (Full n >>= \n -> Full (2 * n)) === Full (2 * n)
+++ OK, passed 100 tests.

### 4. Write a `Monad` Instance for `Reader`

Previously this was marked as "advanced" but this should be pretty old hat now

#### Exercise

In [30]:
instance Monad ((->) r) where
  (>>=) :: (->) r a -> (a -> (->) r b) -> (->) r b
  (>>=) = _bind

#### Try it out

In [47]:
-- >>> ((*) =<< (+10)) 7
-- 119
((+10) >>= (*)) 7

119

#### Tests

In [48]:
readerBindTest :: Property
readerBindTest = forAll arbitrary $ \n ->
  ((+10) >>= (*)) n === n * (n + 10)

runReaderBindTests :: IO ()
runReaderBindTests = runTests [
    ("forall n. ((+10) >>= (*)) n === n * (n + 10)", readerBindTest)
  ]

runReaderBindTests

Testing: forall n. ((+10) >>= (*)) n === n * (n + 10)
+++ OK, passed 100 tests.

### 5. Reimplement `<*>` using only `Monad` Operations (though `fmap/<$>` is allowed)

#### Exercise

In [33]:
(<**>) :: (Monad m) => m (a -> b) -> m a -> m b
(<**>) = _apply

#### Try it out

In [35]:
-- >>> ExactlyOne (+10) <**> ExactlyOne 8
-- ExactlyOne 18
ExactlyOne (+10) <**> ExactlyOne 8

ExactlyOne 18

In [37]:
-- >>> (+1) :. (*2) :. Nil <**> 1 :. 2 :. 3 :. Nil
-- [2,3,4,2,4,6]
((+1) :. (*2) :. Nil) <**> (1 :. 2 :. 3 :. Nil)

[2,3,4,2,4,6]

In [38]:
-- >>> Full (+8) <**> Full 7
-- Full 15
Full (+8) <**> Full 7

Full 15

In [39]:
-- >>> Empty <**> Full 7
-- Empty
Empty <**> Full 7

Empty

In [40]:
-- >>> Full (+8) <**> Empty
-- Empty
Full (+8) <**> Empty

Empty

In [41]:
-- >>> ((+) <**> (+10)) 3
-- 16
((+) <**> (+10)) 3

16

In [42]:
-- >>> ((+) <**> (+5)) 3
-- 11
((+) <**> (+5)) 3

11

In [43]:
-- >>> ((+) <**> (+5)) 1
-- 7
((+) <**> (+5)) 1

7

In [44]:
-- >>> ((*) <**> (+10)) 3
-- 39
((*) <**> (+10)) 3

39

In [45]:
-- >>> ((*) <**> (+2)) 3
-- 15
((*) <**> (+2)) 3

15

#### Tests

In [58]:
applyExactlyOne :: Property
applyExactlyOne = once $
  (ExactlyOne (+10) <**> ExactlyOne 8) === ExactlyOne 18

applyList :: Property
applyList = once $
  (((+1) :. (*2) :. Nil) <**> (1 :. 2 :. 3 :. Nil)) === (2 :. 3 :. 4 :. 2 :. 4 :. 6 :. Nil)

applyOptionalFull :: Property
applyOptionalFull = forAll arbitrary $ \n ->
  (Full (+8) <**> Full n) === Full (n + 8)

applyOptionalEmptyFirst :: Property
applyOptionalEmptyFirst = forAll arbitrary $ \n ->
  (Empty <**> Full n) === Empty

applyOptionalEmptySecond :: Property
applyOptionalEmptySecond = once $
  (Full (+8) <**> Empty) === Empty

applyReaderPlusPlus :: Property
applyReaderPlusPlus =
  let
    pairs :: Gen (Int, Int)
    pairs = do
      x <- arbitrary
      y <- arbitrary
      P.return (x, y)
  in forAll pairs $ \(x, y) ->
    ((+) <**> (+ y)) x === (x + (x + y))
    
applyReaderMultPlus :: Property
applyReaderMultPlus =
  let
    pairs :: Gen (Int, Int)
    pairs = do
      x <- arbitrary
      y <- arbitrary
      P.return (x, y)
  in forAll pairs $ \(x, y) ->
    ((*) <**> (+ y)) x === (x * (x + y))

applyTests :: IO ()
applyTests = runTests [
    ("(ExactlyOne (+10) <**> ExactlyOne 8) === ExactlyOne 18", applyExactlyOne)
  , ("(((+1) :. (*2) :. Nil) <**> (1 :. 2 :. 3 :. Nil)) === (2 :. 3 :. 4 :. 2 :. 4 :. 6 :. Nil)", applyList)
  , ("forall n. (Full (+8) <**> Full n) === Full (n + 8)", applyOptionalFull)
  , ("forall n. (Empty <**> Full n) === Empty", applyOptionalEmptyFirst)
  , ("(Full (+8) <**> Empty) === Empty", applyOptionalEmptySecond)
  , ("forall x, y. (+) <**> (+ y)) x === (x + (x + y))", applyReaderPlusPlus)
  , ("forall x, y. (*) <**> (+ y)) x === (x * (x + y))", applyReaderMultPlus)
  ]

applyTests

Testing: (ExactlyOne (+10) <**> ExactlyOne 8) === ExactlyOne 18
+++ OK, passed 1 tests.
Testing: (((+1) :. (*2) :. Nil) <**> (1 :. 2 :. 3 :. Nil)) === (2 :. 3 :. 4 :. 2 :. 4 :. 6 :. Nil)
+++ OK, passed 1 tests.
Testing: forall n. (Full (+8) <**> Full n) === Full (n + 8)
+++ OK, passed 100 tests.
Testing: forall n. (Empty <**> Full n) === Empty
+++ OK, passed 100 tests.
Testing: (Full (+8) <**> Empty) === Empty
+++ OK, passed 1 tests.
Testing: forall x, y. (+) <**> (+ y)) x === (x + (x + y))
+++ OK, passed 100 tests.
Testing: forall x, y. (*) <**> (+ y)) x === (x * (x + y))
+++ OK, passed 100 tests.

### 6. `join`

Flattens nested structure into a single level structure

#### Exercise

In [59]:
join :: (Monad m) => m (m a) -> m a
join = _join

#### Try it out

In [60]:
-- >>> join ((1 :. 2 :. 3 :. Nil) :. (1 :. 2 :. Nil) :. Nil)
-- [1,2,3,1,2]
join ((1 :. 2 :. 3 :. Nil) :. (1 :. 2 :. Nil) :. Nil)

[1,2,3,1,2]

In [61]:
-- >>> join (Full Empty)
-- Empty
join (Full Empty)

Empty

In [62]:
-- >>> join (Full (Full 7))
-- Full 7
join (Full (Full 7))

Full 7

In [63]:
-- >>> join (+) 7
-- 14
join (+) 7

14

#### Tests

In [69]:
flattenList :: Property
flattenList = once $
  (join ((1 :. 2 :. 3 :. Nil) :. (1 :. 2 :. Nil) :. Nil)) === (1 :. 2 :. 3 :. 1 :. 2 :. Nil)
  
joinOptional1 :: Property
joinOptional1 = once $
  join (Full Empty) === Empty
  
joinOptional2 :: Property
joinOptional2 = forAll arbitrary $ \n ->
  join (Full (Full n)) === Full n
  

joinPlus :: Property
joinPlus = forAll arbitrary $ \n ->
  join (+) n === 2 * n
  
runJoinTests :: IO ()
runJoinTests = runTests [
    ("(join ((1 :. 2 :. 3 :. Nil) :. (1 :. 2 :. Nil) :. Nil)) === (1 :. 2 :. 3 :. 1 :. 2 :. Nil)", flattenList)
  , ("forall n. join (+) n === n + n === 2 * n", joinPlus)
  , ("join (Full Empty) === Empty", joinOptional1)
  , ("forall n. join (Full (Full n)) === Full n", joinOptional2)
  ]

runJoinTests

Testing: (join ((1 :. 2 :. 3 :. Nil) :. (1 :. 2 :. Nil) :. Nil)) === (1 :. 2 :. 3 :. 1 :. 2 :. Nil)
+++ OK, passed 1 tests.
Testing: forall n. join (+) n === n + n === 2 * n
+++ OK, passed 100 tests.
Testing: join (Full Empty) === Empty
+++ OK, passed 1 tests.
Testing: forall n. join (Full (Full n)) === Full n
+++ OK, passed 100 tests.

### 7. Kleisli Composition

Functions of the form `a -> m b` for `m` some `Monad` are referred to as [Kleisli arrows](https://wiki.haskell.org/Arrow_tutorial)

Write a function to compose them. The type of `<=<` also explains the objective, so it can also resolve some
confusion if there is any.

#### Exercise

In [70]:
(<=<) :: (Monad m) => (b -> m c) -> (a -> m b) -> (a -> m c)
(<=<) = _kleisli

#### Try it out

In [71]:
-- >>> ((\n -> n :. n :. Nil) <=< (\n -> n+1 :. n+2 :. Nil)) 1
-- [2,2,3,3]
((\n -> n :. n :. Nil) <=< (\n -> n + 1 :. n + 2 :. Nil)) 1

[2,2,3,3]

#### Tests

In [75]:
kleisliList :: Property
kleisliList = forAll arbitrary $ \m ->
  ((\n -> n :. n :. Nil) <=< (\n -> n + 1 :. n + 2 :. Nil)) m === (m + 1 :. m + 1 :. m + 2 :. m + 2 :. Nil)

kleisliTests :: IO ()
kleisliTests = runTests [
    ("((\\n -> n :. n :. Nil) <=< (\\n -> n + 1 :. n + 2 :. Nil)) m === (m + 1 :. m + 1:. m + 2 :. m + 2 :. Nil)", kleisliList)
  ]

kleisliTests

Testing: ((\n -> n :. n :. Nil) <=< (\n -> n + 1 :. n + 2 :. Nil)) m === (m + 1 :. m + 1:. m + 2 :. m + 2 :. Nil)
+++ OK, passed 100 tests.